<a href="https://colab.research.google.com/github/ryan-miles/stellationharness/blob/main/SynthHRDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

def generate_synthetic_hr_data(num_records=300):
    """
    Generates a synthetic HR dataset.

    Args:
        num_records (int): The number of employee records to generate.

    Returns:
        pandas.DataFrame: A DataFrame containing the synthetic HR data.
    """
    fake = Faker()

    # Define possible values based on observed data and common HR categories
    # Using sets for unique values and then converting to lists for random.choice
    sex_options = ['M', 'F'] # Original data had 'M ' and 'F ' - cleaning to 'M', 'F'
    marital_desc_options = ['Single', 'Married', 'Divorced', 'Widowed', 'Separated']
    citizen_desc_options = ['US Citizen', 'Eligible NonCitizen', 'Non-Citizen']
    hispanic_latino_options = ['No', 'Yes']
    race_desc_options = ['White', 'Black or African American', 'Asian', 'Two or more races', 'Hispanic', 'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander'] # Expanded
    employment_status_options_active = ['Active']
    employment_status_options_term = ['Voluntarily Terminated', 'Terminated for Cause']
    term_reason_options = ['N/A-StillEmployed', 'career change', 'hours', 'return to school', 'attendance', 'performance', 'relocation out of area', 'more money', 'military', 'medical'] # Expanded a bit

    departments = {
        'Production': ['Production Technician I', 'Production Technician II', 'Production Manager'],
        'IT/IS': ['Sr. DBA', 'IT Support', 'Data Analyst', 'Database Administrator', 'Enterprise Architect', 'IT Manager - DB', 'IT Manager - Support', 'IT Manager - Infra', 'Network Engineer'],
        'Software Engineering': ['Software Engineer', 'Sr. Software Engineer', 'Software Engineering Manager'],
        'Admin Offices': ['Administrative Assistant', 'President & CEO', 'Shared Services Manager', 'Accountant I', 'Sr. Accountant', 'CIO', 'Director of Operations'],
        'Sales': ['Sales Manager', 'Area Sales Manager', 'Director of Sales', 'Sales Rep'],
        'Executive Office': ['President & CEO', 'CIO', 'CFO', 'CTO']
    }
    all_positions = [pos for sublist in departments.values() for pos in sublist]

    recruitment_source_options = ['LinkedIn', 'Indeed', 'Google Search', 'Employee Referral', 'Diversity Job Fair', 'On-line Web application', 'CareerBuilder', 'Website', 'Other']
    performance_score_options = ['Exceeds', 'Fully Meets', 'Needs Improvement', 'PIP (Performance Improvement Plan)'] # Ensuring 'PIP' is more descriptive

    # Generate unique manager names
    num_managers = num_records // 10  # Roughly 1 manager for every 10 employees
    manager_names = [fake.name() for _ in range(max(5, num_managers))] # ensure at least 5 managers

    data = []

    for i in range(num_records):
        emp_id = 10000 + i
        employee_name = fake.name()

        dob_date = fake.date_of_birth(minimum_age=18, maximum_age=65)
        # Ensure hire date is at least 18 years after DOB and no later than 30 days ago
        eighteenth_birthday = dob_date + timedelta(days=18*365)
        thirty_days_ago = datetime.now().date() - timedelta(days=30)

        # Ensure the start date for hire_date generation is not after the end date
        # The hire date must be after the 18th birthday and no later than 30 days ago.
        # The start date for `date_between_dates` should be the later of the 18th birthday
        # and the earliest possible hire date (which implicitly must be before the end date).
        # However, simply ensuring the start date is before the end date prevents the ValueError.
        # A robust hire date must be after DOB + 18 years AND at least 30 days before today.
        # Let's pick the maximum of the 18th birthday and a date sufficiently in the past from the end date.
        # A simpler approach is to just make sure the start date is <= the end date.
        # The hire date must be >= 18 years after DOB.
        # The hire date must be <= today - 30 days.
        # Therefore, we need to generate a date between max(DOB + 18 years, earliest_possible_hire_date) and (today - 30 days).
        # The earliest_possible_hire_date constraint is implicitly handled by the end date being today - 30 days.
        # So, we just need to make sure DOB + 18 years is not after today - 30 days.
        # If DOB + 18 years is after today - 30 days, the range is empty.
        # To fix this, the hire date must be AT LEAST DOB + 18 years. If this is after today + 30 days, there's no valid hire date within the constraint of "hired at least 30 days ago".
        # A simpler fix is to just cap the start date at the end date if it's later.
        hire_date_start = dob_date + timedelta(days=18*365)
        hire_date_end = datetime.now().date() - timedelta(days=30)

        # Adjust hire_date_start if it's after hire_date_end
        if hire_date_start > hire_date_end:
             # If the 18th birthday is less than 30 days ago, set the start date to 30 days ago.
             # This ensures the generated hire date is at least 30 days in the past.
             hire_date_start = hire_date_end


        hire_date = fake.date_between_dates(date_start=hire_date_start, date_end=hire_date_end)


        # Termination logic
        termd = random.choice([0, 1]) if (datetime.now().date() - hire_date).days > 90 else 0 # Higher chance of termination if employed longer, not terminated if very new
        if termd == 1 and (datetime.now().date() - hire_date).days < 30 : # Avoid termination if hired in last 30 days
            termd = 0

        date_of_termination = None
        term_reason = 'N/A-StillEmployed'
        employment_status = random.choice(employment_status_options_active)

        if termd == 1:
            # Ensure termination date is after hire date and not in the future
            max_termination_date = datetime.now().date()
            possible_termination_start = hire_date + timedelta(days=30) # Min 30 days employment before termination

            if possible_termination_start > max_termination_date : # If hired very recently, cannot be terminated in the past
                 date_of_termination = possible_termination_start # Terminate on the earliest possible date
            else:
                date_of_termination = fake.date_between_dates(date_start=possible_termination_start, date_end=max_termination_date)

            term_reason = random.choice([r for r in term_reason_options if r != 'N/A-StillEmployed'])
            employment_status = random.choice(employment_status_options_term)

        marital_desc = random.choice(marital_desc_options)
        married_id = 1 if marital_desc == 'Married' else 0
        marital_status_id = marital_desc_options.index(marital_desc)

        sex = random.choice(sex_options)
        gender_id = 0 if sex == 'F' else 1 # Assuming F=0, M=1 as common convention

        dept_name = random.choice(list(departments.keys()))
        position = random.choice(departments[dept_name])

        # Simplified ID mapping for dept, position, performance
        dept_id = list(departments.keys()).index(dept_name) + 1 # 1-based index
        position_id = all_positions.index(position) + 1 # 1-based index

        performance_score = random.choice(performance_score_options)
        perf_score_id = performance_score_options.index(performance_score) + 1 # 1-based index

        # Salary based on rough department/position logic (very simplified)
        salary = 0
        if 'Manager' in position or 'Sr.' in position or 'Director' in position or 'President' in position or 'CIO' in position or 'CFO' in position or 'CTO' in position:
            salary = random.randint(80000, 250000)
        elif 'Technician' in position:
            salary = random.randint(45000, 75000)
        elif 'Engineer' in position or 'Architect' in position or 'DBA' in position:
            salary = random.randint(70000, 150000)
        elif 'Analyst' in position:
            salary = random.randint(60000, 120000)
        elif 'Sales' in position:
             salary = random.randint(50000, 150000)
        else:
            salary = random.randint(40000, 90000)


        last_perf_review_date = None
        if hire_date < (datetime.now().date() - timedelta(days=90)): # Performance review only after 90 days of hire
            # Ensure review date is after hire date and before termination date (if terminated) and before today
            review_start_date = hire_date + timedelta(days=90)
            review_end_date = date_of_termination - timedelta(days=1) if date_of_termination else datetime.now().date()
            if review_start_date < review_end_date:
                 last_perf_review_date = fake.date_between_dates(date_start=review_start_date, date_end=review_end_date)
            elif review_start_date == review_end_date:
                 last_perf_review_date = review_start_date
            else: # if start is after end (e.g. terminated quickly), no sensible review date
                 last_perf_review_date = hire_date # fallback or None


        manager_name = random.choice(manager_names) if dept_name != 'Executive Office' else None # CEO typically doesn't have a manager in this context
        manager_id = manager_names.index(manager_name) + 1 if manager_name else np.nan


        data.append({
            'Employee_Name': employee_name,
            'EmpID': emp_id,
            'MarriedID': married_id,
            'MaritalStatusID': marital_status_id,
            'GenderID': gender_id,
            'EmpStatusID': 0 if employment_status == 'Active' else (1 if employment_status == 'Voluntarily Terminated' else 2), # Example mapping
            'DeptID': dept_id,
            'PerfScoreID': perf_score_id,
            'FromDiversityJobFairID': random.choice([0, 1]),
            'Salary': salary,
            'Termd': termd,
            'PositionID': position_id,
            'Position': position,
            'State': fake.state_abbr(),
            'Zip': fake.zipcode(),
            'DOB': dob_date.strftime('%m/%d/%Y'),
            'Sex': sex,
            'MaritalDesc': marital_desc,
            'CitizenDesc': random.choice(citizen_desc_options),
            'HispanicLatino': random.choice(hispanic_latino_options),
            'RaceDesc': random.choice(race_desc_options),
            'DateofHire': hire_date.strftime('%m/%d/%Y'),
            'DateofTermination': date_of_termination.strftime('%m/%d/%Y') if date_of_termination else np.nan,
            'TermReason': term_reason,
            'EmploymentStatus': employment_status,
            'Department': dept_name,
            'ManagerName': manager_name,
            'ManagerID': manager_id, # Can be float due to NaN
            'RecruitmentSource': random.choice(recruitment_source_options),
            'PerformanceScore': performance_score,
            'EngagementSurvey': round(random.uniform(1.0, 5.0), 2),
            'EmpSatisfaction': random.randint(1, 5),
            'SpecialProjectsCount': random.randint(0, 8),
            'LastPerformanceReview_Date': last_perf_review_date.strftime('%m/%d/%Y') if last_perf_review_date else np.nan,
            'DaysLateLast30': random.randint(0, 10) if employment_status == 'Active' else 0,
            'Absences': random.randint(0, 20) if employment_status == 'Active' else 0
        })

    df = pd.DataFrame(data)

    # Ensure column order matches the original dataset if possible (best effort)
    # Using a predefined list based on common HR dataset structures, as the original order might not be strictly required
    # For robustness, you could pass the original_columns list to this function.
    # For now, use the order as columns are added to the dictionary.

    return df

if __name__ == '__main__':
    # Generate the synthetic dataset
    synthetic_df = generate_synthetic_hr_data(num_records=2000) # Generate 2000 records

    # Save the synthetic dataset to a CSV file
    output_filename = "synthetic_HR_dataset.csv"
    synthetic_df.to_csv(output_filename, index=False)

    print(f"\nSynthetic HR dataset with {len(synthetic_df)} records generated and saved to '{output_filename}'.")
    print("\nFirst 5 rows of the synthetic dataset:")
    print(synthetic_df.head())
    print("\nInformation about the synthetic dataset:")
    synthetic_df.info()


Synthetic HR dataset with 2000 records generated and saved to 'synthetic_HR_dataset.csv'.

First 5 rows of the synthetic dataset:
      Employee_Name  EmpID  MarriedID  MaritalStatusID  GenderID  EmpStatusID  \
0    James Gonzalez  10000          1                1         0            0   
1       Lisa George  10001          0                3         1            0   
2  Gabriel Chandler  10002          0                2         1            0   
3   Melinda Simpson  10003          0                0         0            0   
4          Amy Moss  10004          0                0         1            0   

   DeptID  PerfScoreID  FromDiversityJobFairID  Salary  ...  \
0       5            4                       0  113875  ...   
1       3            4                       0  104243  ...   
2       3            3                       1   89843  ...   
3       5            2                       1   90780  ...   
4       5            3                       1   90110  ...   

   